In [18]:
import tensorflow as tf
import numpy as np
# Be sure it is a jpg

In [66]:
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def load_labels(label_file):
    label = []
    file = open(label_file, "r") 
    proto_as_ascii_lines = file.readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label


In [9]:
def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
    image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)

    return result

In [74]:
graph = load_graph('output_graph.pb')
label_file = load_labels('output_labels.txt')
input_layer = "Placeholder"
output_layer = "final_result"
image_path = "cat-golden.jpg"

input_height = 299
input_width = 299
input_mean = 0
input_std = 255

In [75]:
t = read_tensor_from_image_file(
  image_path,
  input_height=input_height,
  input_width=input_width,
  input_mean=input_mean,
  input_std=input_std)

input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

In [76]:
with tf.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
    results = np.squeeze(results)
    top_k = results.argsort()[-5:][::-1] # Order of the top classes
    labels = label_file
    
    for i in top_k:
        print(labels[i], results[i])      


['0 2', '8 10']
[0 1]
0 2 0.91301507
8 10 0.08698491
